In [ ]:
import tellurium as te
import matplotlib.pyplot as plt
import libsbml

# function for plotting the phase portraits of a given model
# specify metabolites_list to analyze a specific set of metabolites
def plot_phase_portraits(libsbml_model, model_name, simulation, metabolites_list, outpath):
    model_phase_portraits = {}
    used_metabolites = []
    for metabolite in metabolites_list:
        model_phase_portraits[metabolite] = simulation[metabolite]
    for i in metabolites_list:
        for j in metabolites_list:
            if i != j and j not in used_metabolites:
                i_name = libsbml_model.getSpecies(i).getName()
                j_name = libsbml_model.getSpecies(j).getName()
                plt.figure()
                plt.title("Comparison between {} and {} [{}]".format(i_name, j_name, model_name))
                plt.xlabel(i_name)
                plt.ylabel(j_name)
                plt.plot(model_phase_portraits[i], model_phase_portraits[j])
                plt.plot(model_phase_portraits[i][0], model_phase_portraits[j][0], c='r', label='t=0', marker='.')
                plt.plot(model_phase_portraits[i][-1], model_phase_portraits[j][-1], c='k', label='t->inf', marker='.')
                plt.legend()
                plt.savefig(outpath + "_" + i_name + "_vs_" + j_name + ".png")
                plt.show()
        used_metabolites.append(i)


def plot_pool_transient(simulation, title, phase_tuples, outpath):
    pool = 0
    for coefficient, metabolite in phase_tuples:
        pool += coefficient * simulation[metabolite]
    plt.figure()
    plt.title(title)
    plt.xlabel('t')
    plt.plot(pool)
    plt.savefig(outpath)
    plt.show()


def plot_pool_phase_portrait(simulation, title, phase_tuple1, name1, phase_tuple2, name2, outpath):
    pool1 = 0
    for coefficient, metabolite in phase_tuple1:
        pool1 += coefficient * simulation[metabolite]

    pool2 = 0
    for coefficient, metabolite in phase_tuple2:
        pool2 += coefficient * simulation[metabolite]
    plt.figure()
    plt.title(title)
    plt.xlabel(name1)
    plt.ylabel(name2)
    plt.plot(pool1, pool2)
    plt.plot(pool1[0], pool2[0], c='r', label='t=0', marker='.')
    plt.plot(pool1[-1], pool2[-1], c='k', label='t->inf', marker='.')
    plt.savefig(outpath)
    plt.show()


def plot_tiled_phase_portraits(libsbml_model, simulation, components_list, outpath, isReactions):
    model_phase_portraits = {}
    used_components = []
    for component in components_list:
        model_phase_portraits[component] = simulation[component]
    f, axarr = plt.subplots(len(components_list), len(components_list))
    for i in range(len(components_list)):
        for j in range(len(components_list)):
            if i != j and j not in used_components:
                x_component = model_phase_portraits[components_list[i]]
                y_component = model_phase_portraits[components_list[j]]
                axarr[i, j].plot(x_component, y_component)
                axarr[i, j].plot(x_component[0], y_component[0], c='r', label='t=0',
                                 marker='.')  # Red zero, black infty
                axarr[i, j].plot(x_component[-1], y_component[-1], c='k', label='t->inf', marker='.')
    for index, ax in enumerate(axarr.flat):
        x_index = index % len(components_list)
        y_index = index // len(components_list)
        if isReactions:
            xlabel = libsbml_model.getReaction(components_list[x_index]).getId()
            ylabel = libsbml_model.getReaction(components_list[y_index]).getId()
        else:
            xlabel = libsbml_model.getSpecies(components_list[x_index]).getId()
            ylabel = libsbml_model.getSpecies(components_list[y_index]).getId()
        ax.set(xlabel=xlabel, ylabel=ylabel)
        # Hide x labels and tick labels for top plots and y ticks for right plots.
        ax.label_outer()
    plt.savefig(outpath)
    plt.show()


In [3]:
asansm_path = 'xml/iPAE1146_Amino_sugar_and_nucleotide_sugar_metabolism_fbc_squeezed_with_boundaries.xml'
pm_path = 'xml/iPAE1146_Pyrimidine_metabolism_fbc_squeezed_with_boundaries.xml'
lb_path = 'xml/iPAE1146_Lipopolysaccharide_biosynthesis_fbc_squeezed_with_boundaries.xml'
combined_model_path = 'xml/iPAE1146_Combined_Subsystems_fbc_squeezed_with_boundaries.xml'

asansm_closed_path = 'fbc/iPAE1146_Amino_sugar_and_nucleotide_sugar_metabolism_fbc_squeezed.xml'
pm_closed_path = 'fbc/iPAE1146_Pyrimidine_metabolism_fbc_squeezed.xml'
lb_closed_path = 'fbc/iPAE1146_Lipopolysaccharide_biosynthesis_fbc_squeezed.xml'
combined_model_closed_path = 'fbc/iPAE1146_Combined_Subsystems_fbc_squeezed.xml'

asansm_libsbml_doc = libsbml.readSBML(asansm_path)
asansm_libsbml = asansm_libsbml_doc.getModel()
asansm_model = te.loadSBMLModel(asansm_path)
asansm_species = [species.getId() for species in asansm_libsbml.getListOfSpecies()]
asansm_reactions = asansm_model.getReactionIds()

pm_libsbml_doc = libsbml.readSBML(pm_path)
pm_libsbml = pm_libsbml_doc.getModel()
pm_model = te.loadSBMLModel(pm_path)
pm_species = [species.getId() for species in pm_libsbml.getListOfSpecies()]
pm_reactions = pm_model.getReactionIds()

lb_libsbml_doc = libsbml.readSBML(lb_path)
lb_libsbml = lb_libsbml_doc.getModel()
lb_model = te.loadSBMLModel(lb_path)
lb_species = [species.getId() for species in lb_libsbml.getListOfSpecies()]
lb_reactions = lb_model.getReactionIds()

combined_libsbml_doc = libsbml.readSBML(combined_model_path)
combined_libsbml = combined_libsbml_doc.getModel()
combined_model = te.loadSBMLModel(combined_model_path)
combined_species = [species.getId() for species in combined_libsbml.getListOfSpecies()]
combined_reactions = combined_model.getReactionIds()

speciesId2Name = {species.getId(): species.getName() for species in combined_libsbml.getListOfSpecies()}
speciesName2Id = {species.getName(): species.getId() for species in combined_libsbml.getListOfSpecies()}
reactionName2Id = {reaction.getName(): reaction.getId() for reaction in combined_libsbml.getListOfReactions()}
reactionId2Name = {reaction.getId(): reaction.getName() for reaction in combined_libsbml.getListOfReactions()}

# Also load 'closed' models without boundary species to simulate pools better
asansm_model_closed = te.loadSBMLModel(asansm_closed_path)
pm_model_closed = te.loadSBMLModel(pm_closed_path)
lb_model_closed = te.loadSBMLModel(lb_closed_path)
combined_model_closed = te.loadSBMLModel(combined_model_closed_path)


In [ ]:
# ASANSM Simulation

asansm_model.reset()
asansm_simulation = asansm_model.simulate(0, 1000, 1000, asansm_species + asansm_reactions)
# asansm_model.getSteadyStateValues()

# Choose two conservation pools
asansm_conservation_pool1 = [(1, 'NADP'), (1, 'NADPH')]
asansm_conservation_pool1 = [(coeff, speciesName2Id[species]) for coeff, species in asansm_conservation_pool1]

asansm_conservation_pool2 = [(1, 'NAD'), (1, 'H2O'), (1, 'Acetate')]
asansm_conservation_pool2 = [(coeff, speciesName2Id[species]) for coeff, species in asansm_conservation_pool2]

# Plot pools in closed system
asansm_closed_simulation = asansm_model_closed.simulate(0, 100, 100, asansm_species)
plot_pool_transient(asansm_closed_simulation, "[P1]", asansm_conservation_pool1, "plots/asansm_p1_transient.png")
plot_pool_transient(asansm_closed_simulation, "[P2]", asansm_conservation_pool2, "plots/asansm_p2_transient.png")

# plot_pool_phase_portrait(asansm_simulation, "P1 vs. P2 [ASANSM]", asansm_conservation_pool1, "P1",
#                         asansm_conservation_pool2, "P2", "plots/asansm_p1vsp2.png")

# Disturbance from steady state: increase maximal rate of N-acetylglucosamine-6-phosphate deacetylase tenfold, then plot:
# (tiled) phase plane of N-Acetyl-D-glucosamine 6-phosphate vs.
# D-Glucosamine phosphate vs.
# D-Glucosamine1-phosphate vs.
# N-Acetyl-D-glucosamine1-phosphate
# to see how the flux change propagates inside the system
# and a transient response of the fluxes of
# N-acetylgulcosamine-6-phosphate deacetylase,
# phosphoglucosamine mutase,
# glucosamine-1-phosphate N-acetyltransferase

# Perform disturbance
asansm_disturbance_reactionId = reactionName2Id['N-acetylglucosamine-6-phosphate deacetylase']

asansm_disturbance_reactionIndex, asansm_disturbance_reaction = \
    [(index, reaction) for (index, reaction) in enumerate(asansm_libsbml.getListOfReactions()) if
     reaction.getId() == asansm_disturbance_reactionId][0]
asansm_disturbance_reaction_oldFlux = asansm_model.getReactionRates()[asansm_disturbance_reactionIndex]
asansm_disturbance_reaction_params = list(asansm_disturbance_reaction.getListOfAllElements().get(1))
# So now we now which parameters there are - the one we want is vmar_R_rxn01484
# Multiply maximal velocity by 10
asansm_model["vmar_R_rxn01484"] *= 10

# Plot results of disturbance
asansm_disturbance_simulation = asansm_model.simulate(0, 1000, 1000, asansm_species + asansm_reactions)
asansm_disturbance_metabolites = ['N-Acetyl-D-glucosamine 6-phosphate',
                                  'D-Glucosamine phosphate',
                                  'D-Glucosamine1-phosphate',
                                  'N-Acetyl-D-glucosamine1-phosphate']
asansm_disturbance_metabolites = [speciesName2Id[species] for species in asansm_disturbance_metabolites]
plot_tiled_phase_portraits(asansm_libsbml, asansm_disturbance_simulation, asansm_disturbance_metabolites,
                           "plots/asansm_disturbance_metabolites_tiled.png", False)

asansm_disturbance_reactions = ["N-acetylglucosamine-6-phosphate deacetylase",
                                "phosphoglucosamine mutase",
                                "glucosamine-1-phosphate N-acetyltransferase"]
asansm_disturbance_reactions = [reactionName2Id[reaction] for reaction in asansm_disturbance_reactions]
asansm_disturbance_reactions_simulations = [asansm_disturbance_simulation[r] for r in asansm_disturbance_reactions]
for metabolite_simul in asansm_disturbance_reactions_simulations:
    plt.plot(range(len(metabolite_simul)), metabolite_simul)
plt.legend([reactionId2Name[reaction] for reaction in asansm_disturbance_reactions])
plt.xlabel("t")
plt.ylabel("flux")
plt.savefig("plots/asansm_disturbance_reactions_transient.png")
plt.show()

In [3]:
# PM Simulation
pm_model.reset()
pm_simulation = pm_model.simulate(0, 1000, 1000, pm_species + pm_reactions)

# Choose two conservation pools
pm_conservation_pool1 = [(1, 'Ubiquinone-8'), (1, 'Ubiquinol-8')]
pm_conservation_pool1 = [(coeff, speciesName2Id[species]) for coeff, species in pm_conservation_pool1]

pm_conservation_pool2 = [(-1, 'H+'), (-1, "H2O"), (-2, "L-Aspartate"), (1, "trdox"), (1, "L-Glutamate"),
                         (-2, "NH3"), (-1, "PPi"), (-1, "Cytosine"), (-1, "Uracil"), (1, "Orotidylic acid")]
pm_conservation_pool2 = [(coeff, speciesName2Id[species]) for coeff, species in pm_conservation_pool2]

# Plot pools in closed system
pm_closed_simulation = pm_model_closed.simulate(0, 100, 100, pm_species)
plot_pool_transient(pm_closed_simulation, "[P3]", pm_conservation_pool1, "plots/pm_p3_transient.png")
plot_pool_transient(pm_closed_simulation, "[P4]", pm_conservation_pool2, "plots/pm_p4_transient.png")

# Plot the reaction rates of the pathway in PM
pm_pathway_reactions = [(1, 'UTPammonia ligase(ADP-forming)'),
                        (-1, 'nucleoside-diphosphate kinase (ATP:CDP)'),
                        (1, 'ribonucleoside-diphosphate reductase (CDP)'),
                        (1, 'nucleoside-diphosphate kinase (ATP:dCDP)'),
                        (1, 'dCTP deaminase'),
                        (-1, 'nucleoside-diphosphate kinase (ATP:dUDP)'),
                        (-1, 'ribonucleoside-diphosphate reductase (UDP)'),
                        (1, 'nucleoside-diphosphate kinase (ATP:UDP)'),
                        (-1, 'nucleoside-diphosphate kinase (ATP:dTDP)'),
                        (-1, 'thymidine-triphosphatase irreversible')
                        ]
pm_pathway_reactions = [(coeff, reactionName2Id[reaction]) for coeff, reaction in pm_pathway_reactions]
for disturbance_reaction_simulation in [pm_simulation[reaction[1]] for reaction in pm_pathway_reactions]:
    plt.plot(range(len(disturbance_reaction_simulation)), disturbance_reaction_simulation)
plt.legend([reactionId2Name[reaction] for reaction in [r[1] for r in pm_pathway_reactions]], loc=9,
           bbox_to_anchor=(0.5, -0.1))
plt.xlabel("t")
plt.ylabel("flux")
plt.savefig("plots/pm_pathway_reactions_transient.png")
plt.show()

# Disturbance from steady state: Shut off UTPammonia ligase(ADP-forming)
# Question: Does CTP synthase (glutamine) keep the pathway alive?
# Phase plane of UTP vs. CTP,
# transient responses of pathway reactions (so we can compare to first!)

# Perform Disturbance
pm_disturbance_reactionId = reactionName2Id['UTPammonia ligase(ADP-forming)']
pm_disturbance_reactionIndex, pm_disturbance_reaction = [(index, reaction) for (index, reaction) in
                                                         enumerate(pm_libsbml.getListOfReactions())
                                                         if reaction.getId() == pm_disturbance_reactionId][0]
pm_disturbance_reaction_oldFlux = pm_model.getReactionRates()[pm_disturbance_reactionIndex]
pm_disturbance_reaction_params = list(pm_disturbance_reaction.getListOfAllElements().get(1))
# Now we can see that the parameter we want to change is vmar_R_rxn00410, the maximal velocity
# Set maximal velocity to 0 (in theory, this should disable the reaction

pm_model['vmar_R_rxn00410'] = 0
# Mention that the flux changes its steady state value from 1 to 0.6, no idea why it doesn't fall to 0

# Plot results of disturbance
pm_disturbance_simulation = pm_model.simulate(0, 1000, 1000, pm_species + pm_reactions)
plot_phase_portraits(pm_libsbml, "PM", pm_disturbance_simulation,
                     [speciesName2Id[metabolite] for metabolite in ["UTP", "CTP"]],
                     "plots/pm_disturbance_metabolites_phaseplane.png")

for disturbance_reaction_simulation in [pm_disturbance_simulation[reaction[1]] for reaction in pm_pathway_reactions]:
    plt.plot(range(len(disturbance_reaction_simulation)), disturbance_reaction_simulation)
plt.legend([reactionId2Name[reaction] for reaction in [r[1] for r in pm_pathway_reactions]], loc=9,
           bbox_to_anchor=(0.5, -0.1))
plt.xlabel("t")
plt.ylabel("flux")
plt.savefig("plots/pm_disturbance_pathway_reactions_transient.png")
plt.show()

In [3]:
# LB Simulation
lb_model.reset()
lb_simulation = lb_model.simulate(0, 1000, 1000, lb_species + lb_reactions)
# Choose two conservation pools
lb_conservation_pool1 = [(1, '(R)-3-Hydroxydecanoyl-[acyl-carrier protein]'),
                         (1, '2-hydroxydodecanoyl-[acyl-carrier protein]'),
                         (1, 'ACP'), (1, 'D-3-Hydroxydodecanoyl-[acp]'), (1, 'Dodecanoyl-ACP')]
lb_conservation_pool1 = [(coeff, speciesName2Id[species]) for coeff, species in lb_conservation_pool1]

lb_conservation_pool2 = [(1, 'L-Alanine'),
                         (1, 'Pseudomonas aeruginosa LPS core + KDO2-lipidA'),
                         (1, 'Bactoprenyl diphosphate')]
lb_conservation_pool2 = [(coeff, speciesName2Id[species]) for coeff, species in lb_conservation_pool2]

# Plot pools in closed system
lb_closed_simulation = lb_model_closed.simulate(0, 200, 200, lb_species)
plot_pool_transient(lb_closed_simulation, "[P5]", lb_conservation_pool1, "plots/lb_p5_transient.png")
plot_pool_transient(lb_closed_simulation, "[P6]", lb_conservation_pool2, "plots/lb_p6_transient.png")

# Plot transient response of individual metabolites in first pool - they should add up, so their sum stays constant
lb_conservation_metabolites_1 = [metabolite for coeff, metabolite in lb_conservation_pool1]

lb_conservation_metabolites_1_simulations = [lb_closed_simulation[m] for m in lb_conservation_metabolites_1]
for metabolite_simul in lb_conservation_metabolites_1_simulations:
    plt.plot(range(len(metabolite_simul)), metabolite_simul)
plt.plot(range(len(lb_conservation_metabolites_1_simulations[0])), sum(lb_conservation_metabolites_1_simulations))
plt.legend([speciesId2Name[met] for met in lb_conservation_metabolites_1] + ["Concentration Sum"])
plt.xlabel("t")
plt.ylabel("concentration")
plt.savefig("plots/lb_conservation_pool1_metabolites_transient.png")
plt.show()

# Disturbance from steady state: Increase rate of ALL reactions that consume ATP - not done
pass

In [ ]:
# Combined Simulation
combined_model.reset()
combined_model_simulation = combined_model.simulate(0, 1000, 1000, combined_species)
combined_model.plot(combined_model_simulation)

combined_model.getSteadyStateValues()

# plot_phase_portraits(combined_model, combined_model_simulation)

